#Dataset


In [7]:
! pip install pandas matplotlib seaborn spacy re bs4 pickle
# seaborn spacy re bs4 pickle
# ! python -m spacy download en_core_web_md 

ERROR: Could not find a version that satisfies the requirement re (from versions: none)
ERROR: No matching distribution found for re


In [7]:
!pip install seaborn

  Using cached seaborn-0.12.2-py3-none-any.whl (293 kB)
  Using cached matplotlib-3.7.1-cp311-cp311-win_amd64.whl (7.6 MB)
     -------------------------------------- 163.0/163.0 kB 1.1 MB/s eta 0:00:00
  Using cached cycler-0.11.0-py3-none-any.whl (6.4 kB)
  Using cached fonttools-4.39.2-py3-none-any.whl (1.0 MB)
     ---------------------------------------- 55.4/55.4 kB 2.8 MB/s eta 0:00:00
     ---------------------------------------- 2.5/2.5 MB 5.6 MB/s eta 0:00:00
  Using cached pyparsing-3.0.9-py3-none-any.whl (98 kB)



[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: C:\Users\david\AppData\Local\Programs\Python\Python311\python.exe -m pip install --upgrade pip


In [9]:
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import spacy as sp
from sklearn.svm import LinearSVC
from sklearn import metrics
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
# import en_core_web_sm
import re
from bs4 import BeautifulSoup
import bs4 as bs4
from urllib.parse import urlparse
import requests
from collections import Counter
import pickle

In [12]:
data = pd.read_csv("/data/website_classification.csv")
data = data.sample(frac=1, random_state=42).reset_index(drop=True)

FileNotFoundError: [Errno 2] No such file or directory: '/data/website_classification.csv'

In [3]:
data

NameError: name 'data' is not defined

In [ ]:
data = data.drop('Unnamed: 0', axis=1)
data.head()

In [ ]:
#Datos nulos
data.isnull()

In [ ]:
data.website_url.nunique()

In [ ]:
#Eliminamos los URL repetidos
data = data.drop_duplicates(subset='website_url').reset_index(drop=True)
data.shape

In [ ]:
# Create a new column 'category_id' with encoded categories 
data['category_id'] = data['Category'].factorize()[0]
category_id_df = data[['Category', 'category_id']].drop_duplicates()


# Dictionaries for future use
category_to_id = dict(category_id_df.values)
id_to_category = dict(category_id_df[['category_id', 'Category']].values)

# New dataframe
data.head()

In [ ]:
plt.figure(figsize=(15,5))
sns.countplot(x = 'Category', data=data)
plt.xticks(rotation = 90)

# DATA

In [ ]:
x = data['cleaned_website_text']
y = data['Category']

In [ ]:
print(type(x))

In [ ]:
#tranformo la columna categoria para que sea un array y pueda entrenar la red 
#solo uso label encoder aqui porque es para etiquetas no para texto largo
le = LabelEncoder()
le.fit(y)
# le.classes_    me muestra los textos de las clases que transformara
y = le.transform(y)

In [ ]:
nlp = spacy.load("en_core_web_md")

# Crear una matriz de embedding para cada texto en x
x_em = []
for text in x:
    doc = nlp(text)
    embeddings = doc.vector
    x_em.append(embeddings)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x_em, y, test_size=0.33, random_state=42)

# LINEAR SVC


In [ ]:
model_svc = LinearSVC()
history_svc = model_svc.fit(x_train, y_train)
#loss_svc,acc_svc = model_svc.evaluate(y_test,verbose=1)
#print(f'Loss: {loss_svc} - Accuracy: {acc_svc}')

In [ ]:
# file name, I'm using *.pickle as a file extension
filename = "model_save_svc.pickle"

# save model
pickle.dump(model_svc, open(filename, "wb"))

# load model
mod_salvado_svc= pickle.load(open(filename, "rb"))

In [ ]:
print(x_test)

In [ ]:
predicts_svc=model_svc.predict(x_test)
print(len(x_test))
print(f'The accuracy of train model is {metrics.accuracy_score(y_train,model_svc.predict(x_train))}')
print(f'The accuracy of test model is {metrics.accuracy_score(y_test,predicts_svc)}')

In [ ]:
# Obtener los valores únicos de las etiquetas
labels = sorted(data["Category"].unique())

conf_mat=confusion_matrix(y_test, predicts_svc)
# Crear el DataFrame de la matriz de confusión con los labels
conf_mat_df = pd.DataFrame(conf_mat, index=labels, columns=labels)

# Crear el heatmap con los labels personalizados
sns.heatmap(conf_mat_df, annot=True, fmt='d')

# Personalizar los ejes del heatmap
plt.xlabel('Predicción')
plt.ylabel('Etiqueta Real')
plt.show()

In [ ]:
print(classification_report(y_test, predicts_svc))

In [ ]:
print(accuracy_score(y_test, predicts_svc).round(2))

# Web scraping

In [ ]:
class ScrapTool:
    def visit_url(self, website_url):
        '''
        Visit URL. Download the Content. Initialize the beautifulsoup object. Call parsing methods. Return Series object.
        '''
        #headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.71 Safari/537.36'}
        content = requests.get(website_url,timeout=60).content
        
        #lxml is apparently faster than other settings.
        soup = BeautifulSoup(content, "lxml")
        result = {
            "website_url": website_url,
            "website_name": self.get_website_name(website_url),
            "website_text": self.get_html_title_tag(soup)+self.get_html_meta_tags(soup)+self.get_html_heading_tags(soup)+
                                                               self.get_text_content(soup)
        }
        
        #Convert to Series object and return
        return pd.Series(result)
    
    def get_website_name(self,website_url):
        '''
        Example: returns "google" from "www.google.com"
        '''
        return "".join(urlparse(website_url).netloc.split(".")[-2])
    
    def get_html_title_tag(self,soup):
        '''Return the text content of <title> tag from a webpage'''
        return '. '.join(soup.title.contents)
    
    def get_html_meta_tags(self,soup):
        '''Returns the text content of <meta> tags related to keywords and description from a webpage'''
        tags = soup.find_all(lambda tag: (tag.name=="meta") & (tag.has_attr('name') & (tag.has_attr('content'))))
        content = [str(tag["content"]) for tag in tags if tag["name"] in ['keywords','description']]
        return ' '.join(content)
    
    def get_html_heading_tags(self,soup):
        '''returns the text content of heading tags. The assumption is that headings might contain relatively important text.'''
        tags = soup.find_all(["h1","h2","h3","h4","h5","h6"])
        content = [" ".join(tag.stripped_strings) for tag in tags]
        return ' '.join(content)
    
    def get_text_content(self,soup):
        '''returns the text content of the whole page with some exception to tags. See tags_to_ignore.'''
        tags_to_ignore = ['style', 'script', 'head', 'title', 'meta', '[document]',"h1","h2","h3","h4","h5","h6","noscript"]
        tags = soup.find_all(text=True)
        result = []
        for tag in tags:
            stripped_tag = tag.strip()
            if tag.parent.name not in tags_to_ignore\
                and isinstance(tag, bs4.element.Comment)==False\
                and not stripped_tag.isnumeric()\
                and len(stripped_tag)>0:
                result.append(stripped_tag)
        return ' '.join(result)


sp.prefer_gpu()

#anconda prompt ko run as adminstrator and copy paste this:python -m spacy download en
nlp = en_core_web_sm.load()

def clean_text(doc):
    '''
    Clean the document. Remove pronouns, stopwords, lemmatize the words and lowercase them
    '''
    doc = nlp(doc)
    tokens = []
    exclusion_list = ["nan"]
    for token in doc:
        if token.is_stop or token.is_punct or token.text.isnumeric() or (token.text.isalnum()==False) or token.text in exclusion_list :
            continue
        token = str(token.lemma_.lower().strip())
        tokens.append(token)
    return " ".join(tokens) 

In [ ]:
website='https://www.bbc.com/sport/live/football/64405195'
scrapTool = ScrapTool()

web=dict(scrapTool.visit_url(website))
text=(clean_text(web['website_text']))

print(web['website_name']) #Print titulo.

x_em_2 = []
doc2 = nlp(text)
aux = doc2.vector
x_em_2.append(aux)

In [ ]:
print(text)

In [ ]:
print(x_em_2[0].shape)
print(type(x_em_2))

In [ ]:
print(x_em[0].shape)
print(type(x_em))

In [ ]:
predicts_svc_test=model_svc.predict(x_em_2)

In [ ]:
print(predicts_svc_test)

In [ ]:
print(id_to_category[model_svc.predict(x_em_2)[0]])